In [0]:
!pip install --upgrade -q gspread

# SCHOOL SCHEDULING

Programa a seguir realiza o agendamento de horários de disciplinas para determinadas turmas seguindo algumas linhas de critérios. Para a realização deste procedimento, dentro do programa foi implementado algoritmos seguindo a teoria de grafos. 

Dentro da teoria de grafos, utilizamos de coloração de grafos para resolver o problema. Dentro deste assunto utilizamos do algoritmo de DSATUR para encontrar a coloração própria do grafo e realizar também, a partir deste resultado, um algoritmos de melhoria com busca local.

Considerando como arestas uma relação entre professor, turma e disciplina.
Considerando como arestas os conflitos, baseados em algumas exigências previamente definidas.

Mas para realizar tal solução, foi realizado algumas modificações nos algoritmos base propostos por estas teorias, pare que pudesse atender de forma satisfatória as necessidades do cenário em questão.


In [0]:
import sys
import pandas as pd
import time
import plotly.graph_objects as go

filePath = './Escola_A.xlsx'

dataSchool = pd.read_excel(filePath, sheet_name='Dados')
config = pd.read_excel(filePath, sheet_name='Configuracoes')
restrictionsTeachers = pd.read_excel(filePath, sheet_name='Restricao')
restrictionsClass = pd.read_excel(filePath, sheet_name='Restricoes Turma')
preferencesTeachers = pd.read_excel(filePath, sheet_name='Preferencias')

In [0]:

class GlobalData:
    def __init__(self):
        self.settings = schedulesConfig()
        self.colorsSchedules = createColors(self.settings)
        self.colorsUsed = []
        self.matrixRestrictionsTeachers = creatematrixSchedules(restrictionsTeachers)
        self.matrixPreferencesTeachers = creatematrixSchedules(preferencesTeachers)
        self.matrixRestrictionsClasses = creatematrixSchedules(restrictionsClass)
        self.metedPreferences = {}

class Vertex:
    def __init__(self, id, teacher, schoolClass, theme):
        self.id = id
        self.teacher = teacher
        self.schoolClass = schoolClass
        self.theme = theme
        self.restrictionsColors = []

        self.color = 0
        self.adjacents = []
        self.satur = 0
        self.degree = 0
        self.preference = False

    def addAdjacent(self, vertex):
        self.adjacents.append(vertex)
        self.degree += 1

    def getSatur(self):
        return self.satur

    def upSatur(self):
        self.satur += 1

    def downSatur(self):
        self.satur -= 1

    def addRestrictionColor(self, c):
        if c not in self.restrictionsColors:
            self.restrictionsColors.append(c)

    def upSaturAdjacents(self):
        for vertex in self.adjacents:
            vertex.upSatur()

    # Usado exclusivamente por Dsatur
    def setColor(self, c):
        self.color = c
        self.upSaturAdjacents()

# GlobalData
A classe GlobalData é responsável por armazenar os dados extraídos das tabelas e cores usadas

# Vertex 
A classe Vertex possuí dados usados na solução do problema como:
  - Professor
  - Turma
  - Matéria
  - Array de adjacentes
  - Array de restrições
  - Grau
  - Grau de saturação
  - Cor do vértice

A lógica da modelagem dessa classe se baseia muito nos 3 primeiros atributos, os quais foram identificados como células da solução. 

In [0]:

def creatematrixSchedules(dataToBuild):
    matrix = {}

    for _, row in dataToBuild.iterrows():

        if row[0] not in matrix.keys():
            matrix[row[0]] = {}

        if row[2] not in matrix[row[0]].keys():
            matrix[row[0]][row[2]] = []

        if row[1] not in matrix[row[0]][row[2]]:
            matrix[row[0]][row[2]].append(row[1].strftime("%H:%M"))

    return matrix


def schedulesConfig():
    schedulesConf = []
    for _, s in config.iterrows():
        schedulesConf.append(s[0].strftime("%H:%M"))

    return schedulesConf


# Cria matriz de cores
def createColors(schedulesConf):
    colors = {}

    weekDays = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta']
    c = 1
    for day in weekDays:
        for schedule in schedulesConf:
            colors[day + '-' + schedule] = c
            c += 1

    return colors



# Funções auxiliares

Nesse trecho temos funções auxiliares para criar matrizes preferências e de restrições, lista de horários da escola e quais cores representam quais horários. 

In [0]:
def createVertexList():

    vertexList = []
    count = 1
    for _, row in dataSchool.iterrows():
        for _ in range(int(row[3])):
            v = Vertex(count, str(row[2]), row[1], str(row[0]))
            vertexList.append(v)
            count += 1

    return vertexList

# Criação de vértices

Essa função é responsável por gerar a lista de vértices com base nos dados do arquivo de entrada.

In [0]:
def createGraph(data):
    graph = createVertexList()

    # Cria arestas/lista de adjacencia e adiciona a lista de restricoes de cores
    for i in graph:
        # arestas/lista de adjacencia
        for j in graph:
            if i.id != j.id and(i.teacher == j.teacher or i.schoolClass == j.schoolClass):
                i.addAdjacent(j)

        # Restricoes Professores
        if i.teacher in data.matrixRestrictionsTeachers:
            for day in data.matrixRestrictionsTeachers[i.teacher].keys():
                for schedule in data.matrixRestrictionsTeachers[i.teacher][day]:
                    if data.colorsSchedules[day + '-' + schedule] not in i.restrictionsColors:
                        i.addRestrictionColor(data.colorsSchedules[day + '-' + schedule])

        # Restricoes Turmas
        if i.schoolClass in data.matrixRestrictionsClasses:
            for day in data.matrixRestrictionsClasses[i.schoolClass].keys():
                for schedule in data.matrixRestrictionsClasses[i.schoolClass][day]:
                    if data.colorsSchedules[day + '-' + schedule] not in i.restrictionsColors:
                        i.addRestrictionColor(data.colorsSchedules[day + '-' + schedule])

    return graph


# O grafo

A ideia por trás das conexões entre os vértices é de conectar vértices conflituantes. Foram identificados como fatores de conflito vértices com mesmo professor e vértices com mesma turma, pois um professor não pode dar duas aulas ao mesmo tempo nem duas turmas podem ter duas aulas ao mesmo tempo. Dessa forma garantimos que os vértices adjacentes nunca terão as mesmas cores (horários).

No treho acima existe também a montagem das restrições dos vértices com base nas restrições dos professores e da turma.

In [0]:

def getBiggerSaturVertex(graph):
    actual = None

    for ver in graph:
        if ver.color == 0 and (actual is None or ver.satur > actual.satur or (ver.satur == actual.satur and ver.degree > actual.degree)):
            actual = ver
    return actual


def allColorful(graph):
    for vertex in graph:
        if vertex.color == 0:
            return False
    return True


def toColor(vertex, colorsUsed):
    c = 1
    valid = False
    while not valid:
        valid = True
        if c not in vertex.restrictionsColors:
            for adj in vertex.adjacents:
                if adj.color == c:
                    valid = False
                    break
            if valid:
                vertex.setColor(c)
                if c not in colorsUsed:
                    colorsUsed.append(c)

        else:
            valid = False

        c += 1



# DSatur Auxiliares

Nesse trecho temos funções para retornar o vértice com maior saturação, verificar se o grafo está completamente colorido e uma função para colorir o vértice.

Todas essas funções são auxiliares para o algoritmo de DSatur.

In [0]:
def dsatur(graph, colorsUsed):
    while (allColorful(graph) == False):
        vertexBiggerSatur = getBiggerSaturVertex(graph)
        toColor(vertexBiggerSatur, colorsUsed)
    return True

# DSatur Core

A função principal do algoritmo de DSatur que consistem em procurar o vértice com maior grau de saturação e colori-lo com uma das cores disponíveis

In [0]:
def improvement(graph, data):

    reducedColors = []

    while reducedColors != data.colorsUsed:
        reducedColors = data.colorsUsed
        data.colorsUsed = []
        for c in reducedColors:
            for v in graph:
                if v.color == c:
                    i = 1
                    colorful = False
                    while i in data.colorsSchedules.values() and not colorful:
                        if v.color != i and i not in v.restrictionsColors and not v.preference:
                            valid = True

                            for adj in v.adjacents:
                                if adj.color == i:
                                    valid = False
                                    break

                            if valid:
                                v.color = i
                                colorful = True

                            if v.color not in data.colorsUsed:
                                data.colorsUsed.append(v.color)
                        i += 1

# Melhoramento

Esta função executa um algoritmo de melhoria baseado em busca local para tentar reduzir o número de cores do grafo da seguinte forma:
  ```
    Para cada cor usada:
      Para cada vértice dessa cor:
        Para cada cor disponível para esse vértice:
          se for possível trocar:
            troca a cor do vértice
  ```

In [0]:
def setColorsPreferencesBeforeDsatur(graph, data):

    vertexByTecher = {}
    for v in graph:
        if v.teacher in vertexByTecher.keys():
            vertexByTecher[v.teacher].append(v)
        else:
            vertexByTecher[v.teacher] = [v]

    for teacher, vertexList in vertexByTecher.items():
        if teacher in data.matrixPreferencesTeachers.keys():
            if teacher in vertexByTecher.keys():
                preferences = data.matrixPreferencesTeachers[teacher]
                preferencesCount = 0
                preferencesMetedCount = 0
                notColorful = []
                for day in preferences.keys():
                    preferencesCount += len(preferences[day])
                    for schedule in preferences[day]:
                        color = data.colorsSchedules[day+'-'+schedule]
                        colorful = False

                        for v in vertexList:
                            if v.color == 0 and color not in v.restrictionsColors:
                                valid = True

                                for adj in v.adjacents:
                                    if adj.color == color:
                                        valid = False

                                if valid:
                                    v.setColor(color)
                                    v.preference = True
                                    preferencesMetedCount += 1
                                    colorful = True
                                    break

                        if not colorful:
                            notColorful.append('nao coloriu ' + str(color) + ' para ' + teacher)
                if preferencesMetedCount == preferencesCount:
                    data.metedPreferences[teacher] = True
                else:
                    data.metedPreferences[teacher] = False

        else:
            data.metedPreferences[teacher] = True

# Preferências

Buscando atender todas as preferências dos professores, essa função executa uma coloração em alguns vértices antes de executar DSatur. Nota-se que nem sempre será possível atender a preferência e algoritmo de melhoria pode trocar a cor do vértice buscando minimizar os horários usados.

In [0]:
def generateCsv(graph, data):
    schoolClasses = {}
    for v in graph:
        for schedule, color in data.colorsSchedules.items():
            if color == v.color:
                v.schedule = schedule

        if v.schoolClass in schoolClasses.keys():
            schoolClasses[v.schoolClass].append(v)
        else:
            schoolClasses[v.schoolClass] = [v]

    for sc, listSC in schoolClasses.items():
        listSC.sort(key=lambda s: s.color)

        week = {}

        for day in ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta']:
            for scItem in listSC:
                if day in scItem.schedule:
                    if day not in week.keys():
                        week[day] = []
                    week[day].append(scItem)

        for day in week.keys():
            if len(week[day]) < len(data.settings):
                scheduleList = data.settings
                for i in range(len(scheduleList)):
                    foundClass = False
                    for cls in week[day]:
                        if cls is not None and scheduleList[i] in cls.schedule:
                            foundClass = True
                    if not foundClass:
                        week[day].insert(i, None)

        matrixPlot = [data.settings]

        for day in week.keys():
            textList = []
            for cls in week[day]:
                if cls is None:
                    textList.append(None)
                else:
                    textList.append(cls.teacher + ' - Matéria: ' + cls.theme)
            matrixPlot.append(textList)

        header = ["<b>" + str(sc) + "</b>"]
        for day in week.keys():
            header.append(day)

        fig = go.Figure(data=[go.Table(header=dict(values=header), cells=dict(values=matrixPlot))])
        fig.show()

# Saída

Essa função é responsável por escrever o quadro de horários num arquivo .csv para visualização no excel, por exemplo.

In [0]:
def main():
    data = GlobalData()
    graph = createGraph(data)

    setColorsPreferencesBeforeDsatur(graph, data)
    if dsatur(graph, data.colorsUsed):
        improvement(graph, data)
        generateCsv(graph, data)
        print(data.metedPreferences)

        if len(data.colorsSchedules) >= len(data.colorsUsed):
            print('Sucesso')
        else:
            print('Opa, mais cores que o ideal :/')
    else:
        print('Erro ao executar o Dsatur')


startTime = time.time()
main()
print("%s segundos - Tempo de execucao" % (time.time() - startTime))

{'Professor 1': True, 'Professor 2': True, 'Professor 3': True, 'Professor 4': True, 'Professor 5': True, 'Professor 6': True, 'Professor 7': True, 'Professor 8': True, 'Professor 9': False, 'Professor 10': True, 'Professor 11': True, 'Professor 12': True, 'Professor 28': True, 'Professor 13': True, 'Professor 14': True, 'Professor 16': True, 'Professor 15': False, 'Professor 19': True, 'Professor 17': True, 'Professor 18': True, 'Professor 20': True, 'Professor 21': False, 'Professor 22': True, 'Professor 23': True, 'Professor 24': False, 'Professor 25': False, 'Professor 26': True, 'Professor 27': True}
Sucesso
3.0125370025634766 segundos - Tempo de execucao


# Main

Na função principal os métodos são chamados para executar a solução. Primeiramente os dados são extraídos do arquivo de entrada e o grafo é montado.

A função que colore o grafo com as preferências é executado antes do algoritmo de coloração e, após seu sucesso é executada uma função de melhoria para tentar reduzir o numero de cores usadas e, finalmente, a resposta é escrita num arquivo .csv.